Installation

In [1]:
!pip install torch==2.0.1 torchtext==0.15.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 93.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 74.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 9.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 30.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 6.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 9.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install datasets

Import statements

In [3]:
import torchtext
import string
import nltk
import re
import html
import random
import subprocess
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from datasets import load_dataset
from tqdm import tqdm
from collections import defaultdict
import zipfile
import os
import math
from random import shuffle

In [4]:
def split_dataset(file_path, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    sentences = []
    
    with open(file_path, 'r') as f:
        para = ""
        for line in tqdm(f, desc="Splitting dataset"):
            if line.strip():
                para += line.strip() + " "
            else:
                if para:
                    sentences.extend(sent_tokenize(para))
                    para = ""
        if para:
            sentences.extend(sent_tokenize(para))

    shuffle(sentences)

    total_sentences = len(sentences)
    train_size = int(total_sentences * train_ratio)
    val_size = int(total_sentences * val_ratio)
    
    train_sentences = sentences[:train_size]
    val_sentences = sentences[train_size:train_size + val_size]
    test_sentences = sentences[train_size + val_size:]

    return train_sentences, val_sentences, test_sentences

In [5]:
def save_datasets(train_sentences, val_sentences, test_sentences):
    with open('train.txt', 'w') as f:
        f.writelines([s + '\n' for s in train_sentences])
    with open('dev.txt', 'w') as f:
        f.writelines([s + '\n' for s in val_sentences])
    with open('test.txt', 'w') as f:
        f.writelines([s + '\n' for s in test_sentences])

In [6]:
train_sentences, val_sentences, test_sentences = split_dataset('/kaggle/input/dataset/Auguste_Maquet.txt')
save_datasets(train_sentences, val_sentences, test_sentences)

Splitting dataset: 128612it [00:02, 46237.92it/s]


In [7]:
def get_embeddings(emb_file='glove.6B.300d.txt'):
    unk_emb = torch.zeros(300)  # Placeholder embedding for unknown words
    embeddings = defaultdict(lambda: unk_emb)

    with open(emb_file, 'r', encoding='ISO-8859-1') as f:
        for line in tqdm(f, desc="Reading embeddings"):
            try:
                split = line.strip().split()
                word = split[0]
                vector = torch.tensor([float(x) for x in split[1:]])  # Corrected line
                embeddings[word] = vector
            except ValueError as e:
                continue

    return embeddings

In [21]:
embeddings = get_embeddings('/kaggle/input/glove/pytorch/default/1/glove.6B.300d.txt')

Reading embeddings: 400000it [01:13, 5433.58it/s]


In [22]:
class TextData(Dataset):
    def __init__(self, file_path='train.txt', pretrained_emb_dict=embeddings,
                 frequency_cutoff=1, context_size=5, vocab=None):
        self.file_path = file_path
        self.frequency_cutoff = frequency_cutoff
        self.context_size = context_size

        self.contexts = []
        self.words = []

        self.frequency_dictionary = defaultdict(lambda: 0)
        self.vocab = vocab if vocab else []

        self.words2indices = {}
        self.embeddings = pretrained_emb_dict

        with open(self.file_path, 'r') as f:
            for line in tqdm(f, desc="Obtaining vocabulary and freq counts"):
                words = [word.lower() for word in word_tokenize(line)]
                if not vocab:
                    self.vocab += words
                for word in words:
                    self.frequency_dictionary[word] += 1

            if not vocab:
                self.vocab = list(set(self.vocab))
                self.vocab = [word for word in self.vocab if self.frequency_dictionary[word] > self.frequency_cutoff]
                self.vocab.append('<unk>')
            self.words2indices = {w: i for i, w in enumerate(self.vocab)}

        embeddings_list = []
        for word in self.vocab:
            embeddings_list.append(self.embeddings[word])
        embeddings_list.append(self.embeddings['<unk>'])
        self.embeddings = torch.stack(embeddings_list)

        with open(self.file_path, 'r') as f:
            for line in tqdm(f, desc="Creating dataset"):
                words = [word.lower() for word in word_tokenize(line)]
                indices = [self.words2indices[word] if word in self.vocab else (len(self.vocab) - 1)
                           for word in words]
                embeddings = [self.embeddings[i] for i in indices]

                for i in range(len(embeddings) - self.context_size):
                    self.contexts.append(torch.stack(embeddings[i:i + self.context_size]))
                    self.words.append(indices[i + self.context_size])

        self.contexts = torch.stack(self.contexts)
        self.words = torch.tensor(self.words)

    def __getitem__(self, idx):
        return (self.contexts[idx], self.words[idx])

    def __len__(self):
        return len(self.contexts)

In [23]:
train_ds = TextData()
with open('vocab.txt', 'w') as f:
    for word in train_ds.vocab:
        f.write(word + '\n')

test_ds = TextData('test.txt', vocab=train_ds.vocab)
dev_ds = TextData('dev.txt', vocab=train_ds.vocab)

Obtaining vocabulary and freq counts: 39555it [00:09, 4059.13it/s]
Creating dataset: 39555it [02:02, 322.10it/s]
Obtaining vocabulary and freq counts: 5652it [00:01, 4002.83it/s]
Creating dataset: 5652it [00:17, 321.76it/s]
Obtaining vocabulary and freq counts: 11301it [00:02, 3972.54it/s]
Creating dataset: 11301it [00:36, 312.03it/s]


In [35]:
class LSTM_LanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=300, hidden_dim=300, num_layers=2, padding_idx=0):
        super(LSTM_LanguageModel, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.padding_idx = padding_idx
        
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, batch_ctx):
        batch_size = batch_ctx.size(0)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(batch_ctx.device)  # Hidden state
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(batch_ctx.device)  # Cell state

        lstm_out, (hn, cn) = self.lstm(batch_ctx, (h0, c0))  # lstm_out shape: (batch_size, context_size, hidden_dim)

        last_hidden_state = lstm_out[:, -1, :]  # Last time step
        logits = self.fc(last_hidden_state)

        return logits

    def train_epoch(self, dl, optimiser, loss_fn):
        super().train()  # Ensure model is in training mode
        for batch in tqdm(dl):
            optimiser.zero_grad()
            contexts, words = batch

            # Forward pass
            logits = self.forward(contexts)

            # Compute loss, ignoring padding tokens
            loss = loss_fn(logits, words)
            loss.backward()

            optimiser.step()

    def train(self, num_epochs, lr=0.1):
        optimiser = torch.optim.SGD(self.parameters(), lr=lr)
        # Use ignore_index to skip padding tokens in the loss computation
        loss_fn = nn.CrossEntropyLoss(ignore_index=self.padding_idx)
        train_dl = DataLoader(train_ds, batch_size=128)
        dev_dl = DataLoader(dev_ds, batch_size=128)

        for epoch in range(num_epochs):
            print("Epoch:", epoch + 1)
            self.train_epoch(train_dl, optimiser, loss_fn)
            train_loss = self.get_loss(train_dl, loss_fn)
            print("Loss on train set:", train_loss)
            val_loss = self.get_loss(dev_dl, loss_fn)
            print("Loss on validation set:", val_loss)
            train_perp = self.get_perp(train_dl)
            print("Perplexity on train set:", train_perp)
            val_perp = self.get_perp(dev_dl)
            print("Perplexity on validation set:", val_perp)

        return train_perp, val_perp

    def get_loss(self, dl, loss_fn):
        total_loss = 0
        total_samples = 0

        for batch in tqdm(dl):
            contexts, words = batch
            pred = self.forward(contexts)
            loss = loss_fn(pred, words)
            total_loss += loss.item() * len(words)
            total_samples += (words != self.padding_idx).sum().item()  # Exclude padding tokens

        avg_loss = total_loss / total_samples
        return avg_loss

    def get_perp(self, dl, filename='perplexity_output.txt'):
        total_loss = 0
        total_samples = 0
        loss_fn = nn.CrossEntropyLoss(reduction='sum')
        self.eval()

        sentence_perplexities = []
        with open(filename, 'w') as f, torch.no_grad():
            for batch in tqdm(dl):
                contexts, words = batch

                # Forward pass
                pred = self.forward(contexts)

                # Calculate loss for the batch
                loss = loss_fn(pred, words)
                
                # Calculate perplexity for the batch
                perplexity = torch.exp(loss / len(words))

                # Format the contexts (as a sentence) and their corresponding perplexity
                sentence = ' '.join([train_ds.vocab[idx] for idx in words.tolist()])
                f.write(f"{sentence}\t{perplexity.item()}\n")
                
                total_loss += loss.item()
                total_samples += len(words)

                sentence_perplexities.append(perplexity.item())

            avg_loss = total_loss / total_samples
            avg_perplexity = math.exp(avg_loss)

            f.write(f"Average perplexity: {avg_perplexity}\n")
            print(f"Average perplexity: {avg_perplexity}")

        return avg_perplexity


In [12]:
lstm_lm = LSTM_LanguageModel(len(train_ds.vocab))
lstm_lm.train(10)

torch.save(lstm_lm, '10epochs_lstm.pth')

test_dl = DataLoader(test_ds, batch_size=128)
perp = lstm_lm.get_perp(test_dl)
print(perp)

Epoch: 1


100%|██████████| 5012/5012 [01:37<00:00, 51.48it/s]


Loss on train set: 5.747939432516416


100%|██████████| 1452/1452 [00:28<00:00, 51.01it/s]


Loss on validation set: 5.69088221584073


...

Perplexity on train set: 98.30577055114044


100%|██████████| 1452/1452 [00:27<00:00, 52.31it/s]


Perplexity on validation set: 100.70154031545744


100%|██████████| 699/699 [00:13<00:00, 50.76it/s]

99.41911908367919


In [27]:
test_dl = DataLoader(test_ds, batch_size=128)

In [38]:
def experiment(train_ds, dev_ds, test_ds, vocab_size):
    train_dl = DataLoader(train_ds, batch_size=128)
    dev_dl = DataLoader(dev_ds, batch_size=128)
    test_dl = DataLoader(test_ds, batch_size=128)

    hyperparams = [
        {'lr': 0.01, 'hidden_dim': 300, 'num_layers': 2, 'optim': 'SGD'},
        {'lr': 0.001, 'hidden_dim': 400, 'num_layers': 2, 'optim': 'Adam'},
        {'lr': 0.01, 'hidden_dim': 500, 'num_layers': 3, 'optim': 'SGD'}
    ]

    train_perplexities, dev_perplexities, test_perplexities = [], [], []

    for params in hyperparams:
        model = LSTM_LanguageModel(vocab_size, hidden_dim=params['hidden_dim'],
                                   num_layers=params['num_layers'])
        
        print(f"Training with hyperparams: {params}")
        train_perp, val_perp = model.train(num_epochs=10, lr=params['lr'])
        
        train_perplexities.append(train_perp)
        dev_perplexities.append(val_perp)
        
        test_perp = model.get_perp(test_dl)
        test_perplexities.append(test_perp)
        print(f"Test perplexity: {test_perp:.4f}")
        print("==========================")

    return hyperparams, train_perplexities, dev_perplexities, test_perplexities


In [39]:
vocab_size = len(train_ds.vocab)
hyperparams, train_perplexities, dev_perplexities, test_perplexities = experiment(train_ds, dev_ds, test_ds, vocab_size)

Training with hyperparams: {'lr': 0.01, 'hidden_dim': 300, 'num_layers': 2, 'optim': 'SGD'}
Epoch: 1


100%|██████████| 5009/5009 [01:32<00:00, 54.36it/s]


Loss on train set: 6.3540164599362505


100%|██████████| 1437/1437 [00:26<00:00, 53.47it/s]


Loss on validation set: 6.306828181843887


...

Perplexity on train set: 446.0111252009286


100%|██████████| 1437/1437 [01:08<00:00, 21.01it/s]


Perplexity on validation set: 423.58674830915595


100%|██████████| 717/717 [00:34<00:00, 21.08it/s]

Test perplexity: 427.2778


In [ ]:
model = LSTM_LanguageModel(vocab_size=len(train_ds.vocab))

train_dl = DataLoader(train_ds, batch_size=128)
dev_dl = DataLoader(dev_ds, batch_size=128)
test_dl = DataLoader(test_ds, batch_size=128)

train_perplexities, val_perplexities = model.train_model(
    num_epochs=10, 
    lr=0.1, 
    train_dl=train_dl, 
    dev_dl=dev_dl
)

model.get_perp(train_dl, filename='train_perplexity.txt')
model.get_perp(test_dl, filename='test_perplexity.txt')
print("Perplexity files generated for both train and test sets.")